In [22]:
import shutil
import multiprocessing
import ast
import subprocess
import re
import os
import shlex
class log_wrapper:
    def __init__(self, cmdline, tag):
        self.cmdline = cmdline
        self.tag = tag

    def __enter__(self):
        print(f"Running: {self.cmdline}")

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            print(f"Error occurred in {self.tag}: {exc_value}")
        else:
            print(f"{self.tag} completed successfully.")
class SensitivityAnalysis():
    def __init__(self):
        self.file_setup = "setup/YKD-burned_setup.dat"
        self.file_driver = "setup/YKD-burned_driver.dat"
        self.number = 0
        self.target_string = ""
        self.file_list = []
        self.target_values = []
        self.dictionaries = []
        self.list_of_modifies_files = []

    @staticmethod
    def write_dictionary_to_file(dictionary, filename):
        with open(filename, 'w') as file:
            for key, value in dictionary.items():
                file.write(f"{key}\t{value}\n")
            file.write("#\n")

    def create_files(self):
        self.number = len(self.list_of_modifies_files)
        for i, (setup_file, driver_file) in enumerate(self.list_of_modifies_files):
            setup_filename = f"setup/setup_file_{i}.dat"
            driver_filename = f"setup/driver_file_{i}.dat"

            self.write_dictionary_to_file(setup_file, setup_filename)
            self.write_dictionary_to_file(driver_file, driver_filename)

        return self.file_list
    @staticmethod
    def _copy_files(source_file, destination_file):
        with open(source_file, "r") as setup_file, open(destination_file, "w") as new_setup_file:
            shutil.copyfileobj(setup_file, new_setup_file)
    @staticmethod
    
    def process_file(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()

        dictionary = {}
        for line in lines:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            elif line == "end":
                # Include "end" as a key without any associated value
                dictionary["end"] = None
                break  # Stop processing further lines after encountering "end"
            else:
                # Using regular expression to extract key and value parts
                match = re.match(r"^(\S+)\s+(.*)$", line)
                if match:
                    key = match.group(1)
                    value = match.group(2).strip()
                    dictionary[key] = value

        return dictionary

   








    def transform_to_dictionary(self):
        try:
            file_paths = [self.file_setup,self.file_driver]
            with multiprocessing.Pool() as pool:
                self.dictionaries = pool.map(self.process_file, file_paths)
            return self.dictionaries

        except Exception as e:
            raise ValueError(f"Error converting file '{self.file_setup}' or {self.file_driver} to a dictionary: {e}")
    def target_argument(self,target,target_values):
        
        self.target_string = target
        self.target_values = target_values
        self.target_values = target_values
        for dictionary in self.dictionaries:
            temp_list = []

            if self.target_string in dictionary and dictionary ==  self.dictionaries[0]:
                for value in self.target_values:
                    updated_dict = dictionary.copy()
                    updated_dict[self.target_string] = value
                    self.list_of_modifies_files.append((updated_dict,self.dictionaries[1]))

                    
            if self.target_string in dictionary and dictionary == self.dictionaries[1]:
                 for value in self.target_values:
                    updated_dict = dictionary.copy()
                    updated_dict[self.target_string] = value
                    
                    self.list_of_modifies_files.append((self.dictionaries[0],updated_dict))
        return self.list_of_modifies_files
    def run_model(self,run_name):
        program = '/home/kgurbanov/'
        cmdline = program + ' ' + './launch ' + ' ' + run_name

        with log_wrapper(cmdline, tag='run') as lw:
            completed_process = subprocess.run(
                cmdline,             # The program + options
                shell=True,          # must be used if passing options as str and not list
                check=True,          # raise CalledProcessError on failure
                #capture_output=True, # collect stdout and stderr; causes memory problems I think
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                cwd=rundirectory    # control context
            )

            if not completed_process.returncode == 0:
                print(completed_process.stdout)
                print(completed_process.stderr)
       
    


In [21]:
sensitivity = SensitivityAnalysis()
sensitivity.transform_to_dictionary()
sensitivity.target_argument('dataname',["'YKD_01'","'YKD_02'","'YKD_03'"])

sensitivity.create_files()
folders = ["'YKD_01'","'YKD_02'","'YKD_03'"]  # List of run_names to be passed to run_model
rundirectory = '/home/kgurbanov/'  # Define the rundirectory here
with multiprocessing.Pool(processes=(os.cpu_count()-1)) as pool:
      results = pool.map(sensitivity.run_model, folders)

Running: /home/kgurbanov/LAKE-LAKE3.0/ ./launch  'YKD_01'
Running: /home/kgurbanov/LAKE-LAKE3.0/ ./launch  'YKD_02'
Running: /home/kgurbanov/LAKE-LAKE3.0/ ./launch  'YKD_03'
Error occurred in run: Command '/home/kgurbanov/LAKE-LAKE3.0/ ./launch  'YKD_03'' returned non-zero exit status 126.
Error occurred in run: Command '/home/kgurbanov/LAKE-LAKE3.0/ ./launch  'YKD_01'' returned non-zero exit status 126.
Error occurred in run: Command '/home/kgurbanov/LAKE-LAKE3.0/ ./launch  'YKD_02'' returned non-zero exit status 126.


CalledProcessError: Command '/home/kgurbanov/LAKE-LAKE3.0/ ./launch  'YKD_03'' returned non-zero exit status 126.

In [21]:
ls setup/driver_file_0.dat


setup/driver_file_0.dat
